In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()
!conda install -c conda-forge iris

In [ ]:
import glob
import iris
from iris.util import unify_time_units
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import numpy as np
import iris.analysis.cartography
import iris.analysis.stats
import tensorflow as tf
from tensorflow import keras  
from tensorflow.keras.backend import int_shape     
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Add, BatchNormalization, Input, Activation, Cropping2D, Concatenate,ZeroPadding2D
import time
import numpy.ma as ma
import matplotlib.cm as mpl_cm

In [ ]:
cubes_names=[]
directories='/content/drive/MyDrive/U-Birmingham-gpp-data/jja/*BCC*.nc'

for name in glob.glob(directories):
    cubes_names.append(name)
all_cubes = iris.load(cubes_names)
print('Cubes:\n',all_cubes)

print('****************************************************************************************')
  
train_tas,train_pr,train_gpp,test_tas,test_pr,test_gpp,mask_train,mask_test=([] for i in range(8))
train_years = iris.Constraint(year=lambda c: 1850 < c < 2200)
test_years = iris.Constraint(year=lambda c: 2250 < c < 2300)

for x in all_cubes:
  #convert units
    if(x._names[0]=='precipitation_flux'):
       x.convert_units('kg m-2 months-1')
    if(x._names[0]=='air_temperature'):
       x.convert_units('celsius')
    if(x._names[0]=='gross_primary_productivity_of_biomass_expressed_as_carbon'):
       x.convert_units('kg m-2 months-1')
    #scale data   
    try:
        
        if(x._names[0]=='precipitation_flux'):
            print('1')
            data_temp=x.extract(train_years)
            data_temp=(data_temp-data_temp.data.min())/(data_temp.data.max()-data_temp.data.min())
            mask_train.append(data_temp.data._mask)
            train_pr.append(data_temp.data.filled(0))
            data_temp=x.extract(test_years)  
            minimum_pr=data_temp.data.min()
            maximum_pr=data_temp.data.max()  
            mask_test.append(data_temp.data._mask)
            data_temp=(data_temp-data_temp.data.min())/(data_temp.data.max()-data_temp.data.min())            
            test_pr.append(data_temp.data.filled(0))
        if(x._names[0]=='air_temperature'):
            print('2')
            data_temp=x.extract(train_years)
            data_temp=(data_temp-data_temp.data.min())/(data_temp.data.max()-data_temp.data.min())
            train_tas.append(data_temp.data.filled(0))
            data_temp=x.extract(test_years)
            minimum_tas=data_temp.data.min()
            maximum_tas=data_temp.data.max()  
            data_temp=(data_temp-data_temp.data.min())/(data_temp.data.max()-data_temp.data.min())
            test_tas.append(data_temp.data.filled(0))
        if(x._names[0]=='gross_primary_productivity_of_biomass_expressed_as_carbon'):
            print('3')
            data_temp=x.extract(train_years)
            data_temp=(data_temp-data_temp.data.min())/(data_temp.data.max()-data_temp.data.min())
            train_gpp.append(data_temp.data.filled(0))
            data_temp=x.extract(test_years)
            minimum_gpp=data_temp.data.min()
            maximum_gpp=data_temp.data.max()        
            data_temp=(data_temp-data_temp.data.min())/(data_temp.data.max()-data_temp.data.min())
            test_gpp.append(data_temp.data.filled(0))
         
    except:
            print("The variable does not exist")
 
train_pr=np.concatenate(train_pr)               
train_tas=np.concatenate(train_tas)     
train_gpp=np.concatenate(train_gpp)  
mask_train=np.concatenate(mask_train)  
test_tas=np.concatenate(test_tas)     
test_pr=np.concatenate(test_pr)     
test_gpp=np.concatenate(test_gpp)  
mask_test=np.concatenate(mask_test)  
train_data=np.stack([train_tas,train_pr],axis=-1)
test_data=np.stack([test_tas,test_pr],axis=-1)

#The following architecture has been taken from https://github.com/Nishanksingla/UNet-with-ResBlock/blob/master/resnet34_unet_model_2D.py

def res_unet(filter_root, layers, n_class=1, input_size=(64,128,2), activation='relu', batch_norm=True, final_activation='linear'):
    inputs = Input(input_size, name="InputLayer")
    x = inputs
    # Dictionary for long connections
    long_connection_store = {}
    # x=ZeroPadding2D(padding=((3,3),(2,2)))(x)
    # Down sampling
    for i in range(layers):
        out_channel = 2**i * filter_root
        print("out_channel downsampling: {}".format(out_channel))
        # Residual/Skip connection
        res = Conv2D(out_channel, kernel_size=1,padding='same', use_bias=False, name="Identity{}_1".format(i))(x)#defining residual connection
        # First Conv Block with Conv, BN and activation
        conv1 = Conv2D(out_channel, kernel_size=3, padding='same', name="Conv{}_1".format(i))(x)#first convolution
        if batch_norm:
            conv1 = BatchNormalization(name="BN{}_1".format(i))(conv1)#normalize batch
        act1 = Activation(activation, name="Act{}_1".format(i))(conv1)#apply activation
        # Second Conv block with Conv and BN only
        conv2 = Conv2D(out_channel, kernel_size=3, padding='same', name="Conv{}_2".format(i))(act1)#second convolution
        if batch_norm:
            conv2 = BatchNormalization(name="BN{}_2".format(i))(conv2)#second normalization
        resconnection = Add(name="Add{}_1".format(i))([res, conv2])#add the first input of the block with the output of the block
        act2 = Activation(activation, name="Act{}_2".format(i))(resconnection)#apply activation
        # Max pooling
        if i < layers - 1:
            long_connection_store[str(i)] = act2#storing the output of the block in a list
            x = MaxPooling2D(padding='same', name="MaxPooling{}_1".format(i))(act2)#downsampling
        else:
            x = act2
    print("\n")
    # Upsampling
    for i in range(layers - 2, -1, -1):
        print("i upsampling: {}".format(i))
        out_channel = 2**(i) * filter_root
        print("out_channel upsampling: {}".format(out_channel))
        # long connection from down sampling path.
        long_connection = long_connection_store[str(i)]
        print("long_connection: {}".format(long_connection))
        up1 = UpSampling2D(name="UpSampling{}_1".format(i))(x)
        up_conv1 = Conv2D(out_channel, 2, activation='relu', padding='same', name="upsamplingConv{}_1".format(i))(up1)
        print("up_conv1: {}".format(up_conv1))
        crop_shape = get_crop_shape(tf.keras.backend.int_shape(up_conv1), int_shape(long_connection))
        crop_connection = Cropping2D(cropping=crop_shape, name="upCrop{}_1".format(i))(long_connection)
        #  Concatenate.
        up_conc = Concatenate(axis=-1, name="upConcatenate{}_1".format(i))([up_conv1, crop_connection])
        #  Convolutions
        up_conv2 = Conv2D(out_channel, 3, padding='same', name="upConv{}_1".format(i))(up_conc)
        if batch_norm:
            up_conv2 = BatchNormalization(name="upBN{}_1".format(i))(up_conv2)
        up_act1 = Activation(activation, name="upAct{}_1".format(i))(up_conv2)
        up_conv2 = Conv2D(out_channel, 3, padding='same', name="upConv{}_2".format(i))(up_act1)
        if batch_norm:
            up_conv2 = BatchNormalization(name="upBN{}_2".format(i))(up_conv2)
        # Residual/Skip connection
        res = Conv2D(out_channel, kernel_size=1,
                     padding='same', use_bias=False, name="upIdentity{}_1".format(i))(up_conc)
        resconnection = Add(name="upAdd{}_1".format(i))([res, up_conv2])
        x = Activation(activation, name="upAct{}_2".format(i))(resconnection)
    # Final convolution
    # x=Cropping2D(((3,3),(2,2)))(x)
    output = Conv2D(n_class, 1, padding='same',activation=final_activation, name='output')(x)
    return Model(inputs, outputs=output, name='Res-UNet')
# This is not useful, when the input size is not the multiple of 2^layers.
# Dimensions of activation map from down path gets smaller than acitvation map after up sampling operation.
def get_crop_shape(target, source):
    # source is coming from down sampling path.
    # target is coming from up sampling operation.
    source_height_width = np.array(source[1:-1])
    target_height_widht = np.array(target[1:-1])
    diff = (source_height_width - target_height_widht).tolist()
    diff_tup = map(lambda x: (x//2, x//2) if x%2 == 0 else (x//2, x//2 + 1), diff)
    return tuple(diff_tup)

model=res_unet(32,4)
model.summary()
tf.keras.utils.plot_model(model)
opt = keras.optimizers.Adam(learning_rate=1e-5)
l1=tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.NONE)
l2=tf.keras.losses.MeanSquaredError()
model.compile(optimizer=opt,loss=l2,metrics=['accuracy'])

#The following code has been taken from https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/
callbacks = [
     keras.callbacks.ModelCheckpoint(
         filepath="/content/drive/MyDrive/varnita/bcc/jja/resnet/mse",
         save_best_only=True,  # Only save a model if `val_loss` has improved.
         monitor="val_loss",
         verbose=1,
     ),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=5, min_lr=0.0000001),
    keras.callbacks.EarlyStopping(
        # Stop training when `val_loss` is no longer improving
        monitor="val_loss",
        restore_best_weights=True,
        # "no longer improving" being defined as "no better than 1e-2 less"
        min_delta=1e-2,
        # "no longer improving" being further defined as "for at least 2 epochs"
        patience=30,
        verbose=1,
    ),
    keras.callbacks.TensorBoard(
    log_dir="/content/drive/MyDrive/varnita/bcc/jja/resnet/mse",
    histogram_freq=1,  # How often to log histogram visualizations
    embeddings_freq=1,  # How often to log embedding visualizations
    update_freq="epoch",
    )
] 
model.fit({"InputLayer":train_data,},{"output": train_gpp },batch_size=8,epochs=200,callbacks=callbacks,validation_split=0.2)

In [ ]:
#visualizing the output from the model
Model=model( {"InputLayer": test_data} )
matrix=Model.numpy()

num_reshape = np.reshape(matrix, (test_data.shape[0], 64,128))
denorm_data = (num_reshape*(maximum_gpp-minimum_gpp))+minimum_gpp#denormalize data
denorm_data = ma.masked_array(denorm_data,mask_test)# add mask to exclude ocean
ctr=0
brewer_cmap=mpl_cm.get_cmap('brewer_RdYlBu_11')
for x in all_cubes:
        if (x._names[0] == 'gross_primary_productivity_of_biomass_expressed_as_carbon'):
            ground_truth = x.extract(test_years)
            for x in data_temp.slices_over('time'):
               qplt.contourf(x,cmap=brewer_cmap)
               plt.title('GPP expressed as carbon for '+str(x.coord('season_year').points[0]))
               plt.show()
               plt.close("all")
            temp=ground_truth.shape[0]
            estimation= ground_truth.copy()
            estimation.data= denorm_data[ctr:(ctr+ground_truth.shape[0]),:,:]
            ctr+=temp
            for x in estimation.slices_over('time'):
               qplt.contourf(x,cmap=brewer_cmap)
               plt.title('GPP expressed as carbon for '+str(x.coord('season_year').points[0])+ ' estimation')
               plt.show()
               plt.close("all")
            difference=ground_truth-estimation
            for x in difference.slices_over('time'):
                qplt.contourf(x,cmap=brewer_cmap)
                plt.title('Difference between ground truth and estimated GPP for year '+str(x.coord('season_year').points[0]))
                plt.show()
                plt.close("all")


In [ ]:
#for plotting graphs
!pip install tensorboard
%load_ext tensorboard
import tensorflow as tf
import datetime, os
%tensorboard --logdir "/content/drive/MyDrive/varnita/bcc/jja/resnet/mse"